SQL adalah salah satu bahasa populer untuk pemrosesan dan analisis data. Spark mendukung SQL untuk memproses DataFrame. 

Kita akan menggunakan data yang sama dengan yg digunakan pada bab eksplorasi DataFrame.


In [ ]:
%pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 15.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=3b59244c1b16e6f09f1b844a839c9094eded7800a5bf1ede79d3b90e9f244bff
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

Inisialisasi spark session untuk berinteraksi dengan Spark cluster

In [ ]:
spark = SparkSession.builder.appName('DataFrame Basics').getOrCreate()

Download dataset

In [ ]:
!wget https://www.dropbox.com/s/4ozzaa2yvy2kkza/indonesia2013-2015.csv

--2023-02-23 02:35:30--  https://www.dropbox.com/s/4ozzaa2yvy2kkza/indonesia2013-2015.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/4ozzaa2yvy2kkza/indonesia2013-2015.csv [following]
--2023-02-23 02:35:30--  https://www.dropbox.com/s/raw/4ozzaa2yvy2kkza/indonesia2013-2015.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb63717a654a7a8eedbf72fe3fc.dl.dropboxusercontent.com/cd/0/inline/B3AGZKEg431lsP5O07uIuZL4WK_Eb2Dr6oLsMWOF-Y8iJE0GpRJK82iMnR75bcMI59b1S8F07iJtGYAUAcC8yzfzUcyuXzpFb2gcEFpzr981KFSM9tP6zeqAao3418bjmI6-8VaX0u577txrVH-ZUhoEkqxKW1smJfDCwiresMVDMg/file# [following]
--2023-02-23 02:35:31--  https://ucb63717a654a7a8eedbf72fe3fc.dl.dropboxusercontent.com/cd/0/inline/B3AGZKEg431lsP5O07uIuZL4WK_Eb2Dr6oLsMWOF-Y8iJE0GpR

Load ke dataframe

In [ ]:
df = spark.read.csv("indonesia2013-2015.csv",header=True,inferSchema=True)

Sebelum menggunakan SQL, kita perlu membuat temporary table dari dataframe yang akan kita olah.

Gunakan fungsi `createOrReplaceTempView(nama_tabel)` pada dataframe tersebut.

In [ ]:
df.createOrReplaceTempView("penduduk")

Selanjutnya kita bisa menggunakan nama tabel yang sudah kita definisikan dalam SQL statement.

Untuk mengeksekusi SQL statement, kita gunakan fungsi `sql(sqlstatement)` pada spark session.

In [ ]:
spark.sql("select count(*) from penduduk").show()

+--------+
|count(1)|
+--------+
|     102|
+--------+



In [ ]:
spark.sql("select * from penduduk limit 5").show()

+--------------+----+-------+--------+
|      province|year|density|timezone|
+--------------+----+-------+--------+
|          ACEH|2015|     86|     WIB|
|SUMATERA UTARA|2015|    191|     WIB|
|SUMATERA BARAT|2015|    124|     WIB|
|          RIAU|2015|     73|     WIB|
|         JAMBI|2015|     68|     WIB|
+--------------+----+-------+--------+



In [ ]:
spark.sql("""select *, 
     CASE 
      WHEN timezone == 'WIT' THEN 1
      WHEN timezone == 'WITA' THEN 2
      ELSE 3 
     END as kode_timezone  
     from penduduk limit 5""").show()

+--------------+----+-------+--------+-------------+
|      province|year|density|timezone|kode_timezone|
+--------------+----+-------+--------+-------------+
|          ACEH|2015|     86|     WIB|            3|
|SUMATERA UTARA|2015|    191|     WIB|            3|
|SUMATERA BARAT|2015|    124|     WIB|            3|
|          RIAU|2015|     73|     WIB|            3|
|         JAMBI|2015|     68|     WIB|            3|
+--------------+----+-------+--------+-------------+

